In [1]:
import numpy as np

Goal: The goal of the project will be to write a piece of software (possible languages are Python, Julia, and MATLAB), that matches N patients with K doctors. Each patient is allowed to provide a ranked list of their preference for doctors, however doctors are prohibited from displaying preferences for patients. Thus the code should take in the following:

A list of ranked preferences, 1 list for each patient
A maximum capacity for each doctor (can initially assume the same capacity - note the total capacity should exceed the number of patients
And the code should return:

A list of assignments indicating which doctors are to take care of which patients


In [28]:
#A list of ranked preferences, 1 list for each patient
doc_names={0: "A", 1: "B", 2: "C", 3: "D",4:"E"}
preference=np.array([[0,1,2,3,4],
                     [0,3,2,1,4],
                     [1,3,2,4,0],
                     [2,4,3,1,0],
                     [4,3,1,2,0],
                     [3,0,4,1,2]])
patient_num=preference.shape[0]
for i in range(len(preference)):
    pref_individual=preference[i]
    pref_doc_name=[]
    for j in pref_individual:
        pref_doc_name.append(doc_names.get(j))
    pref_list = '>'.join(pref_doc_name)
    print("Patient %d 's preference is %s" % (i, pref_list))

Patient 0 's preference is A>B>C>D>E
Patient 1 's preference is A>D>C>B>E
Patient 2 's preference is B>D>C>E>A
Patient 3 's preference is C>E>D>B>A
Patient 4 's preference is E>D>B>C>A
Patient 5 's preference is D>A>E>B>C


In [3]:
#A maximum capacity for each doctor (can initially assume the same capacity 
#- note the total capacity should exceed the number of patients
doc_capacity=[2,2,1,3,2]

def load_preference(preference_origin, doc_capacity):
    """The capacity can be considered to be equivalent to 
the existence of the multiple same doctors. Therefore,
the columns in the matrix can be multipled according to 
doc_capacity. Pad matrix with max value to achive a square matrix.
    """
    preference = preference_origin.copy()
    hungarian_matrix = preference[:, 0]

    cur_index = 0
    for i in range(len(doc_capacity)):
        capacity = doc_capacity[i]
        preference = np.insert(preference,
                               (capacity - 1) * [cur_index],
                               preference[:, [cur_index]],
                               axis=1)
        cur_index += capacity
    zero_matrix = np.full((preference.shape[1] - preference.shape[0], preference.shape[1]), preference.max(),
                          dtype=int)
    preference = np.vstack((preference, zero_matrix))
    return preference

preference_trans=load_preference(preference, doc_capacity)
print(preference_trans)


[[0 0 1 1 2 3 3 3 4 4]
 [0 0 3 3 2 1 1 1 4 4]
 [1 1 3 3 2 4 4 4 0 0]
 [2 2 4 4 3 1 1 1 0 0]
 [4 4 3 3 1 2 2 2 0 0]
 [3 3 0 0 4 1 1 1 2 2]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]
 [4 4 4 4 4 4 4 4 4 4]]


Hungarian Algorithm
Step 1: Every column and every row subtract its internal minimum
Step 2: Find the row with the fewest zero element
Step 3: Identify the Result
Step 4. Adjust Matrix
Step 5. Calculate the Answer

In [25]:
def mark_min_zero_row(zero_mat, mark_zero):
    """
    The function can be split into two steps:
    #1 The function is used to find the row which containing the fewest 0.
    #2 Select the zero number on the row, and then marked the element corresponding row and column as False
    """

    # find row that has least number of zeros
    min_row = [99999, -1]

    for i in range(zero_mat.shape[0]):
        if np.sum(zero_mat[i] == True) > 0 and min_row[0] > np.sum(zero_mat[i] == True):
            min_row = [np.sum(zero_mat[i] == True),i]


    # Store the marked coordinates into mark_zero
    zero_idxes = np.where(zero_mat[min_row[1],]== True)[0][0]
    mark_zero.append((min_row[1], zero_idxes))

    # Mark the corresponding row and column as False
    zero_mat[min_row[1], :] = False
    zero_mat[:, zero_idxes] = False
    
    return


def mark_matrix(matrix):
    """
    Finding the possible solutions for linear assignment problem.
    """

    # Transform the matrix to boolean matrix(0 = True, others = False)
    cur_matrix = matrix
    zero_bool_matrix = (cur_matrix == 0)
    zero_bool_matrix_copy = zero_bool_matrix.copy()

    # Store possible answer positions by marked_zero
    marked_zero = []  # store [(idxes that marked zero), (row Idx)]
    while True in zero_bool_matrix_copy:
        mark_min_zero_row(zero_bool_matrix_copy, marked_zero)

    # Recording the row and column positions separately.
    marked_zero_row = []
    marked_zero_column = []
    for i in range(len(marked_zero)):
        marked_zero_row.append(marked_zero[i][0])
        marked_zero_column.append(marked_zero[i][1])

    # Store indexes of row that do not contain marked 0 elements
    non_marked_row = list(set(range(cur_matrix.shape[0])) - set(marked_zero_row))

    marked_cols = []
    check_switch = True
    while check_switch:
        check_switch = False
        # Search for any unmarked 0 elements in their column
        for i in range(len(non_marked_row)):
            row_array = zero_bool_matrix[non_marked_row[i], :]
            for j in range(row_array.shape[0]):
                if row_array[j] == True and j not in marked_cols:
                    # Store them in the marked_cols
                    marked_cols.append(j)
                    check_switch = True

        for row_num, col_num in marked_zero:
            # Compare column indexes stored in marked_zero and marked_cols
            if row_num not in non_marked_row and col_num in marked_cols:
                # Save the correspongding row_index into non_marked_rows if a maching column index exists
                non_marked_row.append(row_num)
                check_switch = True
    # The row indexes that are not in non_marked_row are stored in marked_rows
    marked_rows = list(set(range(matrix.shape[0])) - set(non_marked_row))

    return marked_zero, marked_rows, marked_cols


def adjust_matrix(mat, cover_rows, cover_cols):
    """
    1. Find the minimum value that is not in marked_rows and marked_cols
    2. Subtract the elements which are not in marked_rows nor marked_cols form the minimum values
    3. Add the element in marked_rows to the min value
    """

    # Find the minimum value that is not in marked_rows and marked_cols
    min_val = 10000000
    cur_mat = mat
    not_cover_rows = set(range(len(cur_mat))) - set(cover_rows)
    not_cover_cols = set(range(len(cur_mat[0]))) - set(cover_cols)
    for i in not_cover_rows:
        for j in not_cover_cols:
            if min_val > cur_mat[i][j]:
                min_val = cur_mat[i][j]

    # Subtract the elements which are not in marked_rows nor marked_cols from the minimum values
    for i in not_cover_rows:
        for j in not_cover_cols:
            cur_mat[i, j] = cur_mat[i, j] - min_val
    # Add the element in marked_rows to the min value
    for i in range(len(cover_rows)):
        for j in range(len(cover_cols)):
            cur_mat[cover_rows[i], cover_cols[j]] = cur_mat[cover_rows[i], cover_cols[j]] + min_val
    return cur_mat


def hungarian_algorithm(matrix):
    """Return the result of linear assignment from matrix using hungarian algorithm
    """

    cur_matrix = matrix

    # Subtract every column and every row with its internal minimum
    for row_idx in range(matrix.shape[0]):
        min_val = np.min(cur_matrix[row_idx])
        cur_matrix[row_idx] = cur_matrix[row_idx] - min_val
    for col_idx in range(matrix.shape[1]):
        min_val = np.min(cur_matrix[:, col_idx])
        cur_matrix[:, col_idx] = cur_matrix[:, col_idx] - min_val

    zero_count = 0
    # Repeat step 2 and 3 until the zero_count == dimension
    result = None
    dimension = matrix.shape[0]
    while zero_count < dimension:
        # 2 Find the row with the fewest zero elements
        result, marked_rows, marked_cols = mark_matrix(cur_matrix)
        zero_count = len(marked_cols) + len(marked_rows)
        # 3 Identify the result by assessing weather the lengths of marked_rows and marked_cols is equal to the length of the cost matrix
        if zero_count < dimension:
            cur_matrix = adjust_matrix(cur_matrix, marked_rows, marked_cols)

    return result

result = hungarian_algorithm(preference_trans)
result

[(0, np.int64(0)),
 (1, np.int64(1)),
 (2, np.int64(8)),
 (4, np.int64(4)),
 (5, np.int64(2)),
 (3, np.int64(5)),
 (6, np.int64(3)),
 (7, np.int64(6)),
 (8, np.int64(7))]

In [31]:
def transform_result1(result, doc_capacity, doc_names, patient_num):
    """
    Transform 'assignments' into a dictionary (patient: doctor) and remove padding values.
    
    Parameters:
        assignments (list): List of (patient, assigned doctor index) pairs.
        doctor_capacity (list): List of the capacity of each doctor.
        doctor_names (list): List of doctor names corresponding to their indices.
        num_patients (int): Total number of patients to process.
    
    Returns:
        dict: A dictionary mapping patient indices to doctor names.
    """
    # Sort assignments by patient indices
    assignments_sorted = sorted(result, key=lambda x: x[0])

    # Create cumulative capacities for doctor ranges
    cumulative_capacity = [sum(doc_capacity[:i]) for i in range(1, len(doc_capacity) + 1)]
    
    # Create a mapping of patient indices to doctor names
    patient_to_doctor = {}
    for patient_idx, doctor_idx in assignments_sorted:
        # Find the corresponding doctor index using cumulative capacity ranges
        for range_idx, capacity_limit in enumerate(cumulative_capacity):
            if doctor_idx < capacity_limit:  # Doctor index falls in this range
                patient_to_doctor[patient_idx] = doctor_names[range_idx]
                break

    # Ensure patient output maps properly (e.g., removes padding values)
    assignments = {patient_idx: patient_to_doctor[patient_idx] for patient_idx in range(patient_num)}
    return assignment

assignment = transform_result(result, doc_capacity, doc_names, patient_num)
assignment

{0: 'A', 1: 'A', 2: 'E', 3: 'D', 4: 'C', 5: 'B'}

In [36]:
for i in assignment.keys():  
    print("patient {} is taken care by doctor {}".format(str(i), assignment[i]))
       

patient 0 is taken care by doctor A
patient 1 is taken care by doctor A
patient 2 is taken care by doctor E
patient 3 is taken care by doctor D
patient 4 is taken care by doctor C
patient 5 is taken care by doctor B
